## Intrinsic calibration of single camera

Yuchen Yang -- XYZT Lab -- May 29, 2024

In [1]:
import numpy as np
import cv2 as cv
import glob
import os
from matplotlib import pyplot as plt

In [3]:
# Define the dimension of the calibration patterns
gridSize = (22, 16)  # in the format (col, row)    previously (19,14)
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# root_dir = "D:/SAAB/Data/0619_2024/Cam1_23420822"
root_dir = "D:/SAAB/Data/0619_2024/Cam2_23243953"
int_dir = root_dir + "/circleImage1"

if not os.path.exists(int_dir):
    print(f"Make sure the data folder {int_dir} exist")
else:
    print(f"Folder already exist at: {int_dir}")

output_dir = root_dir + "/output"
if not os.path.exists(output_dir):
    # Create the folder
    os.makedirs(output_dir)
    print(f"Folder created at: {output_dir}")
else:
    print(f"Folder already exists at: {output_dir}")

Folder already exist at: D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage
Folder already exists at: D:/SAAB/Data/0619_2024/Cam2_23243953/output


In [4]:
############################################################## Blob detector parameters#######################################################

circleColor = 1 # 1 for white circle and 0 for black circle

# Setup SimpleBlobDetector parameters.
blobParams = cv.SimpleBlobDetector_Params()

# Change thresholds
blobParams.minThreshold = 20.0
blobParams.maxThreshold = 240.0
blobParams.thresholdStep = 5.0
blobParams.minDistBetweenBlobs = 5.0

# Filter by Area.
blobParams.filterByArea = True
blobParams.minArea = 20.0     # minArea may be adjusted to suit for your experiment
blobParams.maxArea = 90000.0   # maxArea may be adjusted to suit for your experiment

# Filter by Circularity
blobParams.filterByCircularity = True
blobParams.minCircularity = 0.4
blobParams.maxCircularity = 1.0
# Filter by Convexity
blobParams.filterByConvexity = True
blobParams.minConvexity = 0.4
blobParams.maxConvexity = 1.0

# Filter by Inertia
blobParams.filterByInertia = False
blobParams.minInertiaRatio = 0.01

blobParams.filterByColor = True
blobParams.blobColor = 255 * circleColor

# Create a detector with the parameters
blobDetector = cv.SimpleBlobDetector_create(blobParams)

In [5]:
objp = np.zeros((gridSize[1]*gridSize[0], 3), dtype = np.float32)
circleCenterDist = 25.4/264 *125 # 125
objp[:, :2] = np.mgrid[0:gridSize[0], 0:gridSize[1]].T.reshape(-1, 2) * circleCenterDist
print(objp)

# for i in range (gridSize[1]):
#     for j in range (gridSize[0]):
#         objp[i, j, 0] = circleCenterDist *i 
#         objp[i, j, 1] = circleCenterDist *j
#         objp[i, j, 2] = 0.0

print(objp.shape)

# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 

# Count the amount of images in the folder
image_extensions = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.tif')
files = os.listdir(int_dir)
images = [file for file in files if file.endswith(image_extensions)]
numPos = len(images)
print("The amount of images for intrinsic calibration: "+ str(numPos))

[[  0.         0.         0.      ]
 [ 13.469697   0.         0.      ]
 [ 26.939394   0.         0.      ]
 [ 40.409092   0.         0.      ]
 [ 53.878788   0.         0.      ]
 [ 67.34849    0.         0.      ]
 [ 80.818184   0.         0.      ]
 [ 94.28788    0.         0.      ]
 [107.757576   0.         0.      ]
 [121.22727    0.         0.      ]
 [134.69698    0.         0.      ]
 [148.16667    0.         0.      ]
 [161.63637    0.         0.      ]
 [175.10606    0.         0.      ]
 [188.57576    0.         0.      ]
 [202.04546    0.         0.      ]
 [215.51515    0.         0.      ]
 [228.98485    0.         0.      ]
 [242.45454    0.         0.      ]
 [  0.        13.469697   0.      ]
 [ 13.469697  13.469697   0.      ]
 [ 26.939394  13.469697   0.      ]
 [ 40.409092  13.469697   0.      ]
 [ 53.878788  13.469697   0.      ]
 [ 67.34849   13.469697   0.      ]
 [ 80.818184  13.469697   0.      ]
 [ 94.28788   13.469697   0.      ]
 [107.757576  13.469697   0.

In [6]:

for i in range(numPos):
    print ("Processing pose " + str(i))
    imagePath = int_dir + "/" + str(i) + ".png"
    print(imagePath)
    RawImg = cv.imread(imagePath, cv.IMREAD_GRAYSCALE)
    # print(RawImg.shape)
    # colorImg = np.zeros_like(RawImg)
    # circleImg = np.zeros_like(RawImg)

    # Check whether the image is read correctly and covert the image in RGB format
    if RawImg is None:
        print ("Error: Image not found")
    else:
        if len(RawImg.shape) == 2:
            print ("The image is grayscale image")
            colorImg = cv.cvtColor(RawImg, cv.COLOR_GRAY2RGB)
            circleImg = cv.cvtColor(colorImg, cv.COLOR_RGB2GRAY)
            print(circleImg.shape)
        elif len(RawImg.shape) == 3:
            print("The image is color image")
            colorImg = cv.cvtColor(RawImg, cv.COLOR_BayerRG2RGB) # The camera is with BayerRG pixel format
            fs = 5.0
            circleImg = cv.cvtColor(colorImg, cv.COLOR_RGB2GRAY) 
        else:
            print("The image format is not recognized")
    
    ######## This is the 
    # cv.namedWindow(' Circle Image ', cv.WINDOW_NORMAL)  # Create a resizable window
    # # Resize the window
    # cv.resizeWindow(' Circle Image ', 800, 600)  # Specify the new size (width, height)
    # cv.imshow(" Circle Image ", RawImg)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    
    # Detect the circle center grid based on the findCirclesGrid
    found, tempCenters = cv.findCirclesGrid(RawImg, gridSize, None , cv.CALIB_CB_CLUSTERING | cv.CALIB_CB_SYMMETRIC_GRID, blobDetector)
    print(found)
    print(tempCenters.shape)

    if found:
        tempLen = len(tempCenters)
        objpoints.append(objp)
        imgpoints.append(tempCenters)

        if gridSize[0]*gridSize[1] == tempCenters.shape[0]:
            visImg = colorImg.copy()
            cv.namedWindow('Detected Circles Grid Pos ' + str(i), cv.WINDOW_NORMAL)  # Create a resizable window
            cv.resizeWindow('Detected Circles Grid Pos ' + str(i), 800, 600)  # Specify the new size (width, height)
            cv.moveWindow('Detected Circles Grid Pos ' + str(i), 100, 100)
            cv.drawChessboardCorners(visImg, gridSize, tempCenters,found)
            cv.imshow('Detected Circles Grid Pos ' + str(i), visImg)
            cv.waitKey(0)
            cv.destroyAllWindows()



Processing pose 0
D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage/0.png
The image is grayscale image
(3000, 4096)
True
(266, 1, 2)
Processing pose 1
D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage/1.png
The image is grayscale image
(3000, 4096)
True
(266, 1, 2)
Processing pose 2
D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage/2.png
The image is grayscale image
(3000, 4096)
True
(266, 1, 2)
Processing pose 3
D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage/3.png
The image is grayscale image
(3000, 4096)
True
(266, 1, 2)
Processing pose 4
D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage/4.png
The image is grayscale image
(3000, 4096)
True
(266, 1, 2)
Processing pose 5
D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage/5.png
The image is grayscale image
(3000, 4096)
True
(266, 1, 2)
Processing pose 6
D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage/6.png
The image is grayscale image
(3000, 4096)
True
(266, 1, 2)
Processing pose 7
D:/SAAB/Data/0619_2024/Cam2_23243953/circleImage/7.png
The

In [7]:
import math
# Calibrate the camera intrinsically and save the corresponding data
print(len(objpoints))
print(len(imgpoints))
print(objpoints[0].shape)
print(imgpoints[0].shape)
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints,RawImg.shape, None, None)

#print("Final reprojection error openCV: ", ret)
# Reporjection error calculation
tot_error = 0
meanError = 0
total_points = 0

for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)

    # print(error)
    tot_error += error
    total_points += len(objpoints[i])

meanError = math.sqrt(tot_error/total_points)
print("Mean reprojection error: {}".format(meanError))

print("Camera matrix : \n")
for row in mtx:
    print(f"{row[0]:.6f}, {row[1]:.6f}, {row[2]:.6f}\n")
print("distortion coefficient : \n")
for coef in dist[0]:
    print(f"{coef:.6f}, ")

IntMatrixFile = output_dir + "/CamIntrinsicMatrix.txt"
with open(IntMatrixFile, "w") as file:
    # file.write(np.array_str(mtx))
    for row in mtx:
        file.write(f"{row[0]:.6f}, {row[1]:.6f}, {row[2]:.6f} \n")

DistVecFile = output_dir + "/CamDistortionCoeff.txt"
with open(DistVecFile, "w") as file1:
    # file1.write(np.array_str(dist))
    for coef in dist[0]:
        file1.write(f"{coef:.6f} ")

40
40
(266, 3)
(266, 1, 2)
Mean reprojection error: 0.2376658728433559
Camera matrix : 

1942.137488, 0.000000, 2092.919377

0.000000, 1938.694362, 1504.359939

0.000000, 0.000000, 1.000000

distortion coefficient : 

-0.052652, 
0.016760, 
0.000403, 
0.000323, 
-0.004408, 
